In [2]:
import numpy as np

In [2]:
# 손실 함수
# 1. MSE
# 2. CEE

In [46]:
#MSE
def mean_squared_error(y, t):
    return 0.5*np.sum((y-t)**2)

In [49]:
t = [0 , 0, 1, 0, 0, 0, 0, 0, 0, 0]

y = [ 0.1 , 0.05 , 0.6 , 0.0 , 0.05 , 0.1 , 0.0 , 0.1 ,0.0, 0.0]
print(mean_squared_error(np.array(y),np.array(t)))

y = [0.1 , 0.05, 0.1, 0.0 , 0.05, 0.1 , 0.0, 0.6, 0.0 , 0.0]
print(mean_squared_error(np.array(y),np.array(t)))

# 손실 함수 쪽 출력이 작으면 정답 레이블과의 오차도 작은 것을 알 수 있다.
# 즉, 평균 제곱 오차(MSE)를 기준으로는 첫 번째 추정 결과가 정답에 더 가까울 것으로 판단할 수 있다.


0.09750000000000003
0.5975


In [60]:
# CrossEntropyError

def cross_entropy_error(y, t):
    delta=1e-7 # 0.0000001
    return -np.sum(t*np.log(y+delta))

In [70]:
t = [0 , 0, 1, 0, 0, 0, 0, 0, 0, 0]

y = [ 0.1 , 0.05 , 0.6 , 0.0 , 0.05 , 0.1 , 0.0 , 0.1 ,0.0, 0.0]
print(cross_entropy_error(np.array(y),np.array(t)))

y = [0.1 , 0.05, 0.1, 0.0 , 0.05, 0.1 , 0.0, 0.6, 0.0 , 0.0]
print(cross_entropy_error(np.array(y),np.array(t)))

0.510825457099338
2.302584092994546


In [116]:
# CEE for Batch
def cross_entropy_error_batch(y, t):
    if y.ndim == 1:
        t = np.reshape(1, t.size)
        y = np.reshape(1, y.size)
    
    batch_size = y.shape[0]
    delta = 1e-7
    return -np.sum(t*np.log(y+delta)) / batch_size
    

In [149]:
y = [[ 0.1 , 0.05 , 0.6 , 0.0 , 0.05 , 0.1 , 0.0 , 0.1 ,0.0, 0.0], 
     [ 0.1 , 0.05 , 0.6 , 0.0 , 0.05 , 0.1 , 0.0 , 0.1 ,0.0, 0.0]]
t = [0 , 0, 1, 0, 0, 0, 1, 1, 0, 0]

cross_entropy_error_batch(np.array(y), np.array(t))

18.9315052010522

In [139]:
# 수치 미분
def numerial_diff(f, x):
    h = 1e-4
    return f(x+h) + f(x-h) / (2*h)

In [172]:
# 편미분과 기울기
def numerical_gradeient(f, x):
    grad = np.zeros_like(x)
    h = 1e-4
    
    for i in range(x.size):
        tmp_val = x[idx]
        
        #f(x+h)
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        #f(x-h)
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        
    return grad
    

In [167]:
#Gradient Descent
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradeient(f, x)
        x -= lr*grad
    return x

In [ ]:
def function(x):
    return x[0]**2 + x[1]**2

init_x = np.array([3.0,4.0])
gradient_descent(function, init_x)

In [182]:
x = [[ 0.1 , 0.05 , 0.6 , 0.0 , 0.05 , 0.1 , 0.0 , 0.1 ,0.0, 0.0], 
     [ 0.1 , 0.05 , 0.6 , 0.0 , 0.05 , 0.1 , 0.0 , 0.1 ,0.0, 0.0]]
x = np.array(x)
it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite']) # 배열 반복
while not it.finished:
    idx = it.multi_index
    print(x[idx])
    
    it.iternext()


0.1
0.05
0.6
0.0
0.05
0.1
0.0
0.1
0.0
0.0
0.1
0.05
0.6
0.0
0.05
0.1
0.0
0.1
0.0
0.0


In [193]:
class simpleNet:
  def __init__(self):
    self.W = np.random.randn(2, 3)

  def predict(self, x):
    return x @ self.W

  def softmax(self, x):
    c = np.max(x)
    exp_x = np.exp(x - c)
    sum_exp_x = np.sum(exp_x)
    return exp_x / sum_exp_x

  def loss(self, x, t):
    z = self.predict(x)
    y = self.softmax(z)
    loss = cross_entropy_error(y, t)
    print(loss)
    return loss

In [184]:
net = simpleNet()
net.W

array([[ 0.64161549, -0.96109082, -0.94285943],
       [-0.74625935,  0.02472272, -0.25724724]])

In [203]:
x = [[ 0.1 , 0.05 , 0.6 , 0.0 , 0.05 , 0.1 , 0.0 , 0.1 ,0.0, 0.0], 
     [ 0.1 , 0.05 , 0.6 , 0.0 , 0.05 , 0.1 , 0.0 , 0.1 ,0.0, 0.0]]
x = np.array(x)
t = [0 , 0, 1, 0, 0, 0, 1, 1, 0, 0]
t = np.array(t)



(0, 0)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 10)

In [212]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

In [219]:
def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x - c)
    sum_exp_x = np.sum(exp_x)
    y = exp_x / sum_exp_x
    return y

In [222]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 정규분포로 초기화

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])

net = simpleNet()
print(net.W)


[[-1.27393504 -0.36551638 -0.4351681 ]
 [ 0.99662755  1.58863322  0.89986101]]
(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)


array([[ 0.11000052,  0.32322264, -0.43322316],
       [ 0.16500078,  0.48483396, -0.64983474]])

In [224]:
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)
dW

(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)


array([[ 0.11000052,  0.32322264, -0.43322316],
       [ 0.16500078,  0.48483396, -0.64983474]])

In [89]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = weight_init_std * np.random.randn(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = weight_init_std * np.random.randn(output_size)
        
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def softmax(self, x):
        c = np.max(x)
        exp_x = np.exp(x-c)
        exp_all = np.sum(exp_x)
        
        return exp_x / exp_all
        
    
    def cross_entropy_error(self, y, t):
       
        if y.ndim == 1:
            t = t.reshape(1, t.size)
            y = y.reshape(1, y.size)
        
        if t.size == y.size:
            t = t.argmax(axis=1)
        
        batch_size = y.shape[0]
        return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size 
            
        
    
    def loss(self, x ,t):
        y = self.predict(x)
        
        return self.cross_entropy_error(y, t)
    
    
    def predict(self, x):
        
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = x @ W1 + b1
        z1 = self.sigmoid(a1)
        a2 = z1 @ W2 + b2
        y = self.softmax(a2)
        
        return y
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def numerical_gradient(self, f, x):
        h = 1e-4
        grad = np.zeros_like(x)
        
        it = np.nditer(x, flags =['multi_index'], op_flags=['readwrite'])
        
        while not it.finished:
            idx = it.multi_index
            tmp_val = x[idx]
            
            x[idx] = tmp_val + h
            fxh1 = f(x)
            
            x[idx]  = tmp_val - h
            fxh2 = f(x)
            
            grad[idx] = (fxh1 - fxh2) / (2*h)
            x[idx] = tmp_val
            
            it.iternext()
            
        return grad
        
    
    def numerical_gradient_all(self, x, t):
        
        loss_W = lambda W: self.loss(x, t)
        
        
        grads = {}
        grads['W1'] = self.numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = self.numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = self.numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = self.numerical_gradient(loss_W, self.params['b2'])
        
        return grad
            
            
            
        

In [90]:
net = TwoLayerNet(input_size = 784, hidden_size = 100, output_size=10)
print(net.params['W1'].shape)  # (784, 100)
print(net.params['b1'].shape)  # (100,)
print(net.params['W2'].shape)  # (100, 10)
print(net.params['b2'].shape)  # (10,)

x = np.random.rand(100, 784)  # 더미 입력 데이터(100장 분량)
# 100 * 100
# (100 * 100) * (100 * 10) => (100, 10)
t = np.random.rand(100, 10)   # 더미 정답 레이블(100장 분량)

grads = net.numerical_gradient_all(x, t)  # 기울기 계산

(784, 100)
(100,)
(100, 10)
(10,)


KeyboardInterrupt: 

In [55]:
print(grads['W1'].shape)  # (784, 100)
print(grads['b1'].shape)  # (100,)
print(grads['W2'].shape)  # (100, 10)
print(grads['b2'].shape)  # (10,)

(784, 100)
(100,)
(100, 10)
(10,)


In [57]:
from dataset03.dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

In [76]:
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

iters_num = 1000  # 반복횟수
train_size = x_train.shape[0]
batch_size = 100  # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iter_per_epoch = max(train_size / batch_size, 1)

batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

grad = network.numerical_gradient_all(x_batch, t_batch)
for key in ('W1', 'b1', 'W2', 'b2'): 
    network.params[key] -= learning_rate * grad[key]
    
loss = network.loss(x_batch, t_batch)
train_loss_list.append(loss)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


array([7, 4, 8, 2, 9, 1, 0, 2, 2, 2, 8, 7, 8, 9, 0, 1, 6, 7, 1, 2, 9, 1,
       9, 1, 6, 1, 7, 2, 4, 5, 6, 7, 0, 1, 6, 0, 6, 6, 3, 3, 7, 8, 8, 2,
       6, 0, 9, 3, 3, 3, 1, 5, 6, 1, 1, 3, 6, 8, 8, 3, 0, 3, 2, 7, 1, 9,
       8, 0, 5, 9, 0, 4, 9, 3, 4, 9, 2, 0, 2, 0, 7, 5, 1, 4, 2, 5, 0, 3,
       3, 8, 8, 8, 8, 2, 7, 5, 0, 0, 4, 8], dtype=uint8)

In [ ]:
from tqdm.auto import tqdm # progress bar

for i in tqdm(range(iters_num)):
    
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]